<a href="https://colab.research.google.com/github/mahitha09/Creditcard-fraud-detection/blob/main/Creditcard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Credit card fraud detection

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

In [25]:
data = pd.read_csv('/content/creditcard.csv')

In [26]:
#First Five rows
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [27]:
#Last Five rows
data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
267331,162725.0,-0.263226,1.082046,-0.369362,-0.713739,0.460032,-0.685490,0.646106,0.284417,-0.180510,-0.648379,-1.635504,0.131118,0.276313,0.375544,-0.352817,0.171224,-0.443409,-0.579266,0.101419,-0.114526,-0.254186,-0.681160,0.056685,-0.720388,-0.381639,0.188822,0.122842,0.027382,8.99,0.0
267332,162726.0,1.934012,-0.214230,-1.727386,-0.129121,1.120018,1.159995,-0.153808,0.306090,0.320286,0.030150,1.326783,1.152779,0.167861,0.704066,0.916965,-0.816365,-0.048060,-0.860067,-0.923047,-0.283394,0.352448,1.231819,0.061903,-0.688903,0.086206,-0.038679,0.025486,-0.077118,1.23,0.0
267333,162726.0,-0.230649,0.315261,-0.541387,-1.494509,3.321725,3.479110,0.470314,0.681980,-0.395329,-0.355487,-0.093495,-0.150251,-0.370865,0.257425,0.385039,-0.486631,-0.354466,-0.764742,0.439586,0.114883,-0.241149,-0.633826,-0.017286,0.704977,-0.474717,0.230399,-0.060045,-0.074326,1.98,0.0
267334,162728.0,1.957555,-0.629307,-0.635363,0.177278,-0.359772,0.307457,-0.744206,0.150395,1.090287,0.168022,-0.214193,0.651319,0.124613,-0.095184,-0.263046,0.818792,-1.094300,0.840794,0.396508,-0.094511,0.163200,0.591334,0.001014,-0.980263,-0.190775,0.609385,-0.030715,-0.062151,39.99,0.0
267335,162728.0,1.972115,-0.161916,-0.962841,0.102880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267336 entries, 0 to 267335
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    267336 non-null  float64
 1   V1      267336 non-null  float64
 2   V2      267336 non-null  float64
 3   V3      267336 non-null  float64
 4   V4      267336 non-null  float64
 5   V5      267335 non-null  float64
 6   V6      267335 non-null  float64
 7   V7      267335 non-null  float64
 8   V8      267335 non-null  float64
 9   V9      267335 non-null  float64
 10  V10     267335 non-null  float64
 11  V11     267335 non-null  float64
 12  V12     267335 non-null  float64
 13  V13     267335 non-null  float64
 14  V14     267335 non-null  float64
 15  V15     267335 non-null  float64
 16  V16     267335 non-null  float64
 17  V17     267335 non-null  float64
 18  V18     267335 non-null  float64
 19  V19     267335 non-null  float64
 20  V20     267335 non-null  float64
 21  V21     26

Data Analysis

In [29]:
# Get the Fraud and the normal dataset 

fraud_data = data[data['Class']==1]

normal_data = data[data['Class']==0]

In [30]:
print(fraud_data.shape,normal_data.shape)


(480, 31) (266855, 31)


In [31]:
fraud_data.Amount.describe()

count     480.000000
mean      121.408896
std       257.964187
min         0.000000
25%         1.000000
50%         8.590000
75%       105.080000
max      2125.870000
Name: Amount, dtype: float64

In [32]:
normal_data.Amount.describe()


count    266855.000000
mean         89.583518
std         248.365742
min           0.000000
25%           5.990000
50%          22.720000
75%          79.000000
max       19656.530000
Name: Amount, dtype: float64

Train and test data

In [33]:
# Take some sample of the data
data_sample= data.sample(frac = 0.1,random_state=1)
data_sample.shape

(26734, 31)

In [34]:
data.shape

(267336, 31)

In [35]:
#Determine the number of fraud and valid transactions in the dataset

Fraud = data_sample[data_sample['Class']==1]
Valid = data_sample[data_sample['Class']==0]

outlier_fraction = len(Fraud)/float(len(Valid))
print(outlier_fraction)

print("Fraud Cases : {}".format(len(Fraud)))
print("Valid Cases : {}".format(len(Valid)))

0.0016485575121768454
Fraud Cases : 44
Valid Cases : 26690


ML MODEL

In [36]:
#Create independent and Dependent Features
columns = data_sample.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["Class"]]
# Store the variable we are predicting 
target = "Class"
# Define a random state 
state = np.random.RandomState(42)
X = data_sample[columns]
Y = data_sample[target]
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

(26734, 30)
(26734,)


In [37]:
#Define the outlier detection methods

classifiers = {
    "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X), 
                                       contamination=outlier_fraction,random_state=state, verbose=0),
    "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto', 
                                              leaf_size=30, metric='minkowski',
                                              p=2, metric_params=None, contamination=outlier_fraction)
}

In [38]:
n_outliers = len(Fraud)
for i, (clf_name,clf) in enumerate(classifiers.items()):
    #Fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_prediction = clf.negative_outlier_factor_
    else:    
        clf.fit(X)
        scores_prediction = clf.decision_function(X)
        y_pred = clf.predict(X)
    #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1
    n_errors = (y_pred != Y).sum()
    # Run Classification Metrics
    print("{}: {}".format(clf_name,n_errors))
    print("Accuracy Score :")
    print(accuracy_score(Y,y_pred))
    print("Classification Report :")
    print(classification_report(Y,y_pred))

Isolation Forest: 63
Accuracy Score :
0.9976434502880227
Classification Report :
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     26690
         1.0       0.29      0.30      0.29        44

    accuracy                           1.00     26734
   macro avg       0.64      0.65      0.65     26734
weighted avg       1.00      1.00      1.00     26734

Local Outlier Factor: 89
Accuracy Score :
0.9966709059624448
Classification Report :
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     26690
         1.0       0.00      0.00      0.00        44

    accuracy                           1.00     26734
   macro avg       0.50      0.50      0.50     26734
weighted avg       1.00      1.00      1.00     26734

